# Entrega YOLO + OCR

In [7]:
from ultralytics import YOLO
import cv2
import numpy as np
import csv

In [21]:
# Open CSV file for writing
csv_file = open('detections.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Write header
csv_writer.writerow([
    'fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking',
    'x1', 'y1', 'x2', 'y2', 'matrícula_en_su_caso', 'confianza_matricula',
    'mx1', 'my1', 'mx2', 'my2', 'texto_matricula'
])

TEST_VIDEO_PATH = "test-video.MP4"
yolo_11_model = YOLO('yolo11n.pt')
license_plate_model = YOLO('license-plate-model/weights/best.pt')

LICENSE_PLATE_SIZE = 640
MAX_CARS_PER_FRAME = 5

classNames = ["person", "bicycle", "car"]

test_video = cv2.VideoCapture(TEST_VIDEO_PATH)

# Get video properties
fps = int(test_video.get(cv2.CAP_PROP_FPS))
width = int(test_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(test_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create VideoWriter object
fourcc = cv2.VideoWriter.fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

# Check if VideoWriter opened successfully
if not out.isOpened():
    print("Error: Could not open VideoWriter")
    exit()
frame_count = 0
has_frame, frame = test_video.read()
while has_frame:
    frame_count += 1
    results = yolo_11_model.track(source=frame, persist=True, verbose=False, classes=[0,1,2], device=0, conf=0.4)
    if results and len(results) > 0:
        annotated_frame = results[0].plot()
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        confs = results[0].boxes.conf.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy().astype(int)


        track_ids = []
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        else:
            track_ids = [-1] * len(boxes)

        car_crops = []
        offsets = []
        car_info = []

        for idx, (box, conf, cls, track_id) in enumerate(zip(boxes, confs, classes, track_ids)):
            x1, y1, x2, y2 = box
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            object_type = classNames[cls]

            if cls in [0, 1]:
                csv_writer.writerow([
                    frame_count, object_type, f"{conf:.3f}", track_id,
                    x1, y1, x2, y2,
                    'No', '', '', '', '', '', ''
                ])
            else: # CAR
                original_crop = frame[y1:y2, x1:x2]
                car_crops.append(cv2.resize(original_crop, (LICENSE_PLATE_SIZE, LICENSE_PLATE_SIZE)))
                offsets.append((x1, y1, x2, y2))

                car_info.append({
                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                        'conf': conf,
                        'id': track_id
                    })

        if car_crops:
            lp_results = license_plate_model.track(source=car_crops, verbose=False, classes=[0], device=0, conf=0.25)
            lp_confs = lp_results[0].boxes.conf.cpu().numpy()

            for i, res in enumerate(lp_results):
                car = car_info[i]
                if len(res.boxes) == 0:
                    # No plate detected
                    csv_writer.writerow([
                        frame_count, 'car', f"{car['conf']:.3f}", car['id'],
                        car['x1'], car['y1'], car['x2'], car['y2'],
                        'No', '', '', '', '', '', ''
                    ])
                elif len(res.boxes) > 0:
                    lp_boxes = res.boxes.xyxy.cpu().numpy()

                    x1, y1, x2, y2 = offsets[i]
                    orig_width = x2 - x1
                    orig_height = y2 - y1

                    scale_x = orig_width / LICENSE_PLATE_SIZE
                    scale_y = orig_height / LICENSE_PLATE_SIZE

                    for lp_box, lp_conf in zip(lp_boxes, lp_confs):
                        lx1, ly1, lx2, ly2 = lp_box
                        abs_lx1 = int(round(lx1 * scale_x + x1))
                        abs_ly1 = int(round(ly1 * scale_y + y1))
                        abs_lx2 = int(round(lx2 * scale_x + x1))
                        abs_ly2 = int(round(ly2 * scale_y + y1))

                        abs_lx1 = max(0, min(abs_lx1, frame.shape[1]))
                        abs_ly1 = max(0, min(abs_ly1, frame.shape[0]))
                        abs_lx2 = max(0, min(abs_lx2, frame.shape[1]))
                        abs_ly2 = max(0, min(abs_ly2, frame.shape[0]))

                        cv2.rectangle(annotated_frame, (abs_lx1, abs_ly1), (abs_lx2, abs_ly2), (0, 0, 255), 2)
                        plate_text = ""  # Replace with OCR result

                        # Write to CSV
                        csv_writer.writerow([
                            frame_count, 'car', f"{car['conf']:.3f}", car['id'],
                            car['x1'], car['y1'], car['x2'], car['y2'],
                            'Sí', f"{lp_conf:.3f}",
                            abs_lx1, abs_ly1, abs_lx2, abs_ly2,
                            plate_text
                        ])

        #cv2.imshow("License Plate YOLO", annotated_frame)
        out.write(annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    has_frame, frame = test_video.read()

test_video.release()
out.release()
cv2.destroyAllWindows()